In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os
import pickle
import copy

import time
from datetime import timedelta

'''
각 레이어에서 '훈련 피처 맵'과 '정분류 테스트 피처 맵'과 '오분류 테스트 피처 맵'은 모두 같은 형태이어야 한다.
예를 들어, 첫번째 레이어에서 훈련 피처 맵(100, 12, 12) 정분류 테스트 피처 맵(30, 12, 12), 오분류 테스트 피처 맵(20, 12, 12)
와 같이 인덱스 첫번째인 개수는 다르더라도 이차 넘파이 배열은 같아야 한다.
'''

class FMD():
    # directory
    # root_dir: 관련된 데이터가 모두 저장되어 있는 디렉토리 경로
    # origin_dir: train, rtest, wtest가 있는 디렉토리
    # eval_dir: 거리를 잴 데이터가 있는 디렉토리
    root_dir=""; origin_dir=""; eval_dir=""

    # data_infos
    # 데이터 타입에 대한 이름
    origin_names = ["train", "rtest", "wtest"]
    eval_names = []
    # (이하) 대괄호로 쳐져있는 배열은 나중에 넘파이 배열이 될 수 있다.
    # (origin_, eval_)K: 피처 맵의 개수, L: 레이어의 개수(0~L-1는 각 레이어의 인덱스)
    # shape: 레이어의 넘파이 모양
    # 아래 (origin_, eval_)K, L, shape을 'data_infos'라고 하자.
    # min max은 정규화된 데이터의 최소값과 최대값을 의미한다.
    origin_K={"train": 0, "rtest": 0, "wtest": 0}; eval_K={}; L=0; shape=[]
    
    # FM_means
    # (이하) 훈련 데이터, 정분류 테스트 데이터, 오분류 테스트 데이터에서 데이터라는 용어를 생략하고 적었다.
    # 아래의 배열들은 모두 파이썬 배열이고 첫 번째 인자에 layer가 들어간다.
    # TFM_mean: 훈련 평균 피처 맵(베이스 피처 맵)
    # RFM_mean: 정분류 테스트 평균 피처 맵
    # WFM_mean: 오분류 테스트 평균 피처 맵
    TFM_mean=[]; RFM_mean=[]; WFM_mean=[]

    # AMs와 그와 관련된 것
    # TAM: 훈련 활성화 피처 맵       , RAM: 정분류 테스트 활성화 피처 맵
    # WAM: 오분류 테스트 활성화 피처 맵
    TAM=[]; RAM=[]; WAM=[]
    # alpha는 거리 계산을 위한 인덱스를 고르기 위해 필요한 변수이다.
    # w_minus_r_max(= max(w-r))는 나중에 alpha 값에 의존하여 최대로 만들어진다.
    # alpha_min, alpha_max는 각 레이어에서의 alpha가 가질 수 있는 최소값 최대값을 나타낸 것이다.
    # [hyper parameter] alpha_slice은 alpha_min에서 alpha_max로 몇 번의의 간격으로 도착할지 알려주는 변수임.
    alpha=[]; alpha_slice=[]; w_minus_r_max=[]
    alpha_min=[]; alpha_max=[]
    # DAM_indexes는 나중에 거리 계산할 때 쓰이는 다차원 인덱스들의 집합이다.
    # 각 원소는 피처 맵의 한 원소의 인덱스를 나타낸다.
    # DAM: 거리 활성화 맵, DAM는 거리 계산을 위한 인덱스만 활성된 맵이다.
    # 각 레이어마다 튜플들 세트가 있어야 함. np.array의 item 메소드를 사용할 것이기 때문
    # [hyper parameter] DAM_select는 DAM를 고르는 방법을 알려줌.
    DAM_indexes=[]; DAM=[]; DAM_select=[]

    # index_infos
    '''
    사용하지 않음.
    '''
    # '오분류 테스트' 각 인덱스에서의 거리에 대한 중간값, 평균, 표준편차, 최소값, 최대값에 대한 배열은
    # 각 mid, mean, std, min, max임
    # 인덱스마다 달라지는 값이니 총칭해서 'index_infos'라고 하자.
    mid=[]; mean=[]; std=[]; min=[]; max=[]
    
    # layer_infos
    '''
    사용하지 않음.
    lfmd_select: 레이어 피처맵 거리 계산 방법
    [hyper parameter] lfmd_select는 각 레이어에 대한 피처 맵을 구하는 방법을 저장한다.
    '''
    lfmd_select=[]
    # W: 레이어 피처맵 거리 가중치
    # [hyper parameter] W는 각 레이어의 피처 맵에 곱할 weight 중요도이다
    # 레이어 피처 맵에 대한 가시적인 출력을 위해 1차 이상의 배열을 2차원 배열로 바꿈
    # square_FM_side은 1차 이상의 배열을 2차원 배열로 바꾸기 위해 필요한 2차원 배열의 한 변의 길이임.
    square_FM_side=[]; W=[]

    # fmdc
    # fmdc 피처 맵 거리 기준으로 어떤 데이터가 나중에 오분류 될 거 같은지 판단함.
    # fmdc는 거리이므로 음수가 될 수 없음, 따라서, fmdc가 -1이라는 것은 아직 계산하지 않았다는 의미임.
    fmdc=-1

    def __init__(self, root_dir_=""):
        # root 디렉토리 입력이 없다면 return
        if root_dir_=="":
            print("루트 디렉토리 경로를 설정해주세요.")
            return
        # root 디렉토리
        self.root_dir = root_dir_

        # origin 디렉토리
        self.origin_dir = f"{self.root_dir}/origin"
        # 훈련을 저장하는 디렉토리
        self.train_dir=f"{self.origin_dir}/{self.origin_names[0]}"
        # 정분류 테스트를 저장하는 디렉토리
        self.rtest_dir=f"{self.origin_dir}/{self.origin_names[1]}"
        # 오분류 테스트를 저장하는 디렉토리
        self.wtest_dir=f"{self.origin_dir}/{self.origin_names[2]}"
        
        # eval 디렉토리
        self.eval_dir = f"{self.root_dir}/eval"

        # os.paht.isdir는 '\ ' 대신, ' '을 써도 됨
        is_there_instances = os.path.isdir(f"{self.root_dir}/instances")
        # 객체를 불러오거나 저장할 디렉토리 생성
        # ' '을 '\ '로 바꿈
        root_dir = self.root_dir.replace(' ', '\ ')
        # instances 디렉토리가 없을 경우만 instances 생성
        if not is_there_instances:
            os.system(f"mkdir {root_dir}/instances")

    def fit(self):
        # 객체의 속성 초기화
        # root_dir의 데이터가 올바르다면 init이 에러 없이 완료됨
        self.set_data_infos()
        self.set_FM_means()
        self.set_AMs()
        # self.set_index_infos()
        self.set_fmdc()

    def set_root_dir(self, root_dir_):
        self.root_dir = root_dir_

        # origin 디렉토리
        self.origin_dir = f"{self.root_dir}/origin"
        # 훈련을 저장하는 디렉토리
        self.train_dir=f"{self.origin_dir}/{self.origin_names[0]}"
        # 정분류 테스트를 저장하는 디렉토리
        self.rtest_dir=f"{self.origin_dir}/{self.origin_names[1]}"
        # 오분류 테스트를 저장하는 디렉토리
        self.wtest_dir=f"{self.origin_dir}/{self.origin_names[2]}"
        
        # eval 디렉토리
        self.eval_dir = f"{self.root_dir}/eval"

    def create_practice(self):
        '''
        아마도 나중에 adapter를 만들 때 사용될 수 있을 것 같음.
        '''

        # 연습 데이터 랜덤 시드 설정
        random_seed_id = 42
        np.random.seed(random_seed_id)
        # 연습 데이터 범위 설정
        '''
        연습 데이터 범위가 0~1로로 고정, np.random.rand(or random)으로 바로 구현 가능
        '''
        # random_start = 1; random_end = 2

        # 디렉토리 설정
        root_dir = "practice"
        origin_dir = f"{root_dir}/origin"
        eval_dir = f"{root_dir}/eval"
        # 디렉토리 생성
        os.system(f"mkdir {root_dir}")
        os.system(f"mkdir {origin_dir}")
        os.system(f"mkdir {eval_dir}")

        # origin 데이터 종류 적기
        origin_names="train rtest wtest"
        # origin_K 각 데이터 종류마다의 개수
        origin_K="10000 2000 4000"
        # eval 데이터 종류 적기
        eval_names="rotation_90 brightness_10 whiteness_10"
        # eval_K 각 데이터 종류마다의 개수
        eval_K="12000 4000 6000"
        # L: 레이어의 개수, shape: 레이어의 모양
        L="3";
        shape = "12 12" + "\n"
        shape += "12 2 3 4" + "\n"
        shape += "24 8"

        # data_infos.txt 파일로 저장
        # origin 데이터 종류는 train rtest wtest로 이미 정해져 있으므로 파일에 저장하지 않음
        os.system(f"touch {root_dir}/data_infos.txt")
        data_infos_txt = open(f"{root_dir}/data_infos.txt", 'w')
        data_infos_txt.write(origin_K + "\n")
        data_infos_txt.write(eval_names + "\n")
        data_infos_txt.write(eval_K + "\n")
        data_infos_txt.write(L + "\n")
        data_infos_txt.write(shape)
        data_infos_txt.close()

        # 자료구조를 바꾸어 for문 활용을 쉽게 하기
        origin_names = origin_names.split()
        origin_K = list(map(int,origin_K.split()))
        origin_K_list = origin_K
        origin_K = {}
        for i in range(len(origin_names)):
            origin_K[origin_names[i]] = origin_K_list[i]
        
        eval_names = eval_names.split()
        eval_K = list(map(int,eval_K.split()))
        eval_K_list = eval_K
        eval_K = {}
        for i in range(len(eval_names)):
            eval_K[eval_names[i]] = eval_K_list[i]

        L=int(L)
        
        shape = shape.split('\n')
        for ith in range(len(shape)):
            shape[ith] = list(map(int,shape[ith].split()))

        # origin 연습 데이터 생성
        for origin_name in origin_names:
            os.system(f"mkdir {origin_dir}/{origin_name}")
            # datas: 데이터들을 최대 1000개씩 담음.
            # file_index: 0 ~ K-1 까지 파일을 1000개씩 담을건데 0에서부터 시작해서 1000개마다 1 증가함
            datas = []; file_index = 0

            for k in range(origin_K[origin_name]):
                # data: 데이터는 레이어들을 담고 있음
                data = []
                # data에 레이어들을 담은 후
                for l in range(L):
                    lth_layer = np.random.random(shape[l])
                    data.append(lth_layer)
                # datas에 레이어들을 담은 data를 담음
                datas.append(data)

                # 1000개마다 데이터가 저장됨
                if len(datas) == 1000:
                    # file_index번째 datas를 저장
                    with open(f'{origin_dir}/{origin_name}/{origin_name}_{file_index}.pickle', 'wb') as f:
                        pickle.dump(datas, f, pickle.HIGHEST_PROTOCOL)
                    # datas의 데이터를 램에서 제거
                    del datas
                    # datas를 빈 배열로 만듦
                    datas = []
                    # 파일 인덱스 1 증가
                    file_index += 1

            # datas의 데이터가 남아있다면, 1~999개의 데이터가 남아있다면 그 데이터도 마저 저장함
            if len(datas) != 0:
                with open(f'{origin_dir}/{origin_name}/{origin_name}_{file_index}.pickle', 'wb') as f:
                    pickle.dump(datas, f, pickle.HIGHEST_PROTOCOL)

            # 훈련 데이터의 경우
            # 정분류(1), 오분류(0) 여부를 알려주는 넘파이 배열을 파일로 저장함
            if origin_name == "train":
                origin_result = np.random.randint(0,2, origin_K[origin_name])
                # 1 -> True, 0 -> False로 변경
                origin_result = np.array(origin_result, dtype="bool")
                np.save(f"{origin_dir}/{origin_name}/{origin_name}_result.npy", origin_result)

        # eval 연습 데이터 생성
        for eval_name in eval_names:
            os.system(f"mkdir {eval_dir}/{eval_name}")
            # datas: 데이터들을 최대 1000개씩 담음.
            # file_index: 0 ~ K-1 까지 파일을 1000개씩 담을건데 0에서부터 시작해서 1000개마다 1 증가함
            datas = []; file_index = 0

            for k in range(eval_K[eval_name]):
                # data: 데이터는 레이어들을 담고 있음
                data = []
                # data에 레이어들을 담은 후
                for l in range(L):
                    lth_layer = np.random.random(shape[l])
                    data.append(lth_layer)
                # datas에 레이어들을 담은 data를 담음
                datas.append(data)

                # 1000개마다 데이터가 저장됨
                if len(datas) == 1000:
                    # file_index번째 datas를 저장
                    with open(f'{eval_dir}/{eval_name}/{eval_name}_{file_index}.pickle', 'wb') as f:
                        pickle.dump(datas, f, pickle.HIGHEST_PROTOCOL)
                    # datas의 데이터를 램에서 제거
                    del datas
                    # datas를 빈 배열로 만듦
                    datas = []
                    # 파일 인덱스 1 증가
                    file_index += 1

            # datas의 데이터가 남아있다면, 1~999개의 데이터가 남아있다면 그 데이터도 마저 저장함
            if len(datas) != 0:
                with open(f'{eval_dir}/{eval_name}/{eval_name}_{file_index}.pickle', 'wb') as f:
                    pickle.dump(datas, f, pickle.HIGHEST_PROTOCOL)

            # 정분류(1), 오분류(0) 여부를 알려주는 넘파이 배열을 파일로 저장함
            eval_result = np.random.randint(0,2, eval_K[eval_name])
            # 1 -> True, 0 -> False로 변경
            eval_result = np.array(eval_result, dtype="bool")
            np.save(f"{eval_dir}/{eval_name}/{eval_name}_result.npy", eval_result)

    def set_data_infos(self):
        # root dir의 data_infos.txt 열기
        data_infos = open(f"{self.root_dir}/data_infos.txt", 'r')
        data_infos_strs = data_infos.read()
        data_infos_str_list = data_infos_strs.split('\n')
        # 0th: origin_K 
        origin_K = list(map(int, data_infos_str_list[0].split()))
        origin_name_K_zip = zip(self.origin_names, origin_K) 
        for origin_name, origin_K in origin_name_K_zip:
            self.origin_K[origin_name] = origin_K
        # 1th: eval_names
        self.eval_names = data_infos_str_list[1].split()
        # 2th: eval_K
        eval_K = list(map(int, data_infos_str_list[2].split()))
        eval_name_K_zip = zip(self.eval_names, eval_K) 
        for eval_name, eval_K in eval_name_K_zip:
            self.eval_K[eval_name] = eval_K
        # 3th: L
        self.L = int(data_infos_str_list[3])
        # 4+0th ~ 4+(L-1)th: shape
        for l in range(self.L):
            shape_l = list(map(int,data_infos_str_list[4+l].split()))
            self.shape.append(shape_l)
        # root dir의 data_infos.txt 닫기
        data_infos.close()

        # 레이어 피처 맵을 구하는 방식을 se_lfmd 모두 통일
        '''
        for l in range(self.L):
            self.lfmd_select.append("se_lfmd")
        '''
        # W를 균등하게 만듦
        for l in range(self.L):
            self.W.append(1/self.L)

    def set_FM_means(self):
        # 인스턴스 속성을 변수로 포인터처럼 가르킴
        train = self.origin_names[0]; rtest = self.origin_names[1]; wtest = self.origin_names[2]
        L = self.L; shape = self.shape

        def set_FM_mean(origin):
            # 인스턴스 속성을 변수로 포인터처럼 가르킴
            origin_K = self.origin_K[origin]

            if origin == train:
                OFM_mean = self.TFM_mean; origin_dir = self.train_dir;
            elif origin == rtest:
                OFM_mean = self.RFM_mean; origin_dir = self.rtest_dir;
            elif origin == wtest:
                OFM_mean = self.WFM_mean; origin_dir = self.wtest_dir;
            else:
                print('잘못된 origin: ', origin, sep='')
                return

            # 각 레이어의 피처 맵을 0으로 초기화하여 생성
            for l in range(L):
                OFM_mean_l = np.zeros(shape[l])
                OFM_mean.append(OFM_mean_l)

            # OFMP_k는 k번째 origin 데이터가 속한 FMP임
            # k번 째 데이터의 OFMPI는 OFMPI_k = k // 1000임
            # k번 째 데이터의 OFMPO은 OFMPO_k = k % 1000임
            OFMP_k = None; prev_OFMPI_k = None; cur_OFMPI_k = None

            k = 0
            # 0번 째 데이터가 속한 OFMP_k를 불러들인 후
            prev_OFMPI_k = k // 1000
            OFMPO_k = k % 1000
            with open(f'{origin_dir}/{origin}_{prev_OFMPI_k}.pickle', 'rb') as f:
                OFMP_k = pickle.load(f)
            # 0번 째 데이터를 OFM_mean에 넣는다.
            for l in range(L):
                OFM_k_l = OFMP_k[OFMPO_k][l]
                OFM_mean[l] = OFM_mean[l] + OFM_k_l

            # k = 1 ~ K-1
            # 1~K-1번 째 데이터로 OFM_mean을 구한다.
            for k in range(1, origin_K):
                # k번 째 데이터의 OFMPI, OFMPO 구함
                cur_OFMPI_k = k // 1000
                OFMPO_k = k % 1000

                # OFMP_k가 이미 있다면 가지고 오지 않고
                #             없다면 이전 OFMP를 지우고 현재 OFMP를 가지고 온다.

                # cur_OFMPI_k와 prev_OFMPI_k가 같다면
                if cur_OFMPI_k == prev_OFMPI_k:
                    pass # 아무 작업 하지 않고

                # cur_OFMPI_k와 prev_OFMPI_k가 다를 경우
                else:
                    # 이전 OFMP_k의 기억공간을 램에서 제거한 후
                    del OFMP_k
                    # cur_OFMPI_k를 현재 OFMP_k를 가지고 온다.
                    with open(f'{origin_dir}/{origin}_{cur_OFMPI_k}.pickle', 'rb') as f:
                        OFMP_k = pickle.load(f)

                # prev_OFMPI_k를 cur_OFMPI_k로 초기화 
                prev_OFMPI_k = cur_OFMPI_k

                for l in range(L):
                    OFM_k_l = OFMP_k[OFMPO_k][l]
                    OFM_mean[l] = (OFM_mean[l]*k + OFM_k_l)/(k+1)

        # 훈련, 정분류 테스트, 오분류 테스트 데이터에 대한 FM_mean을 구함
        set_FM_mean(train); set_FM_mean(rtest); set_FM_mean(wtest)
        
    def set_AMs(self):
        # alpha, w_minus_r_max를 0으로 초기화
        self.alpha = np.zeros(self.L)
        self.w_minus_r_max = np.zeros(self.L)

        # [hyperparameter] alpha_slice를 일단 1000으로 함
        for l in range(self.L):
            self.alpha_slice.append(1000)

        # alpha의 최소값과 최대값을 구함
        self.alpha_min = np.zeros(self.L)
        self.alpha_max = np.zeros(self.L)
        for l in range(self.L):
            self.alpha_min[l] = np.array([self.RFM_mean[l].min(),
                                         self.TFM_mean[l].min(),
                                         self.WFM_mean[l].min()]).min()
            self.alpha_max[l] = np.array([self.RFM_mean[l].max(),
                                         self.TFM_mean[l].max(),
                                         self.WFM_mean[l].max()]).max()

        # TAM, RAM, WAM을 0으로 초기화 함
        for l in range(self.L):
            TAM_l = np.zeros(self.shape[l])
            self.TAM.append(TAM_l)
        for l in range(self.L):
            RAM_l = np.zeros(self.shape[l])
            self.RAM.append(RAM_l)
        for l in range(self.L):
            WAM_l = np.zeros(self.shape[l])
            self.WAM.append(WAM_l)

        # w-r가 최대가 되는 alpha, TAM, RAM, WAM을 찾음
        for l in range(self.L):
            # range 부분 고칠 필요가 있음
            a_min_l = self.alpha_min[l]; a_max_l = self.alpha_max[l]
            a_slice_l = self.alpha_slice[l]
            interval_l = (a_max_l - a_min_l)/a_slice_l
            # range(a_slice_l+1) 해야 a_min_l 부터 a_max_l 까지 감
            for alpha_l in [a_min_l + interval_l*s for s in range(a_slice_l+1)]:
                TAM_l = np.array(self.TFM_mean[l] > alpha_l)
                RAM_l = np.array(self.RFM_mean[l] > alpha_l)
                WAM_l = np.array(self.WFM_mean[l] > alpha_l)
                
                TAM_l_xor_RAM_l = np.logical_xor(TAM_l, RAM_l)
                TAM_l_xor_WAM_l = np.logical_xor(TAM_l, WAM_l)
                # r_l은 TAM_l과 RAM_l이 얼마나 유사하지 않은지 보여준다.
                # 즉, TAM_l과 RAM_l이 유사하지 않을수록 r_l 값이 커진다.
                # w_l도 마찬가지이다.
                r_l = len(np.where(TAM_l_xor_RAM_l == True)[0])
                w_l = len(np.where(TAM_l_xor_WAM_l == True)[0])
                
                if w_l - r_l > self.w_minus_r_max[l]:
                    # w-r가 이전의 w-r보다 클 때
                    # 즉, TAM과 RAM이 더 유사해지거나 TAM과 WAM이 더 다를 때
                    # alpha, w-r, TAM, RAM, WAM를 최신화함.
                    self.w_minus_r_max[l] = w_l - r_l
                    self.alpha[l] = alpha_l
                    self.TAM[l] = TAM_l
                    self.RAM[l] = RAM_l
                    self.WAM[l] = WAM_l

        # TAM, RAM, WAM를 이용하여 DAM를 구함
        # DAM에 WAM를 deep copy 복사함
        for l in range(self.L):
            DAM_l = self.WAM[l].copy()
            self.DAM.append(DAM_l)

        # [hyperparameter] 일단 모든 레이어를 'and' 방식으로 함
        for l in range(self.L):
            self.DAM_select.append("and")

        # 선택하려는 방식('and', 'or', 등등)대로 DAM를 고름
        for l in range(self.L):
            # 'and' 방식보다 'or' 방식에서 대부분의 인덱스에서 오분류 데이터의 값이 상대적으로 더 커짐
            # 다만 'or' 방식은 'and' 방식보다 거리 계산을 위한 인덱스의 개수가 더 줄어듦
            # 'all' 방식은 모든 인덱스를 다 사용함.
            if self.DAM_select[l] == "and":
                TAM_l_and_RAM_l = np.logical_and(self.TAM[l], self.RAM[l])
                # WAM에서 (TAM 교집합 RAM)과 곂치는 부분을 False로 만듦
                np.place(self.DAM[l], TAM_l_and_RAM_l, False)
            elif self.DAM_select[l] == "or":
                TAM_l_or_RAM_l = np.logical_or(self.TAM[l], self.RAM[l])
                # WAM에서 (TAM 합집합 RAM)과 곂치는 부분을 False로 만듦)
                np.place(self.DAM[l], TAM_l_or_RAM_l, False)
            elif self.DAM_select[l] == "all":
                not_DAM_l = np.logical_not(self.DAM[l])
                # 모든 인덱스를 다 사용함
                np.place(self.DAM[l], not_DAM_l, True)

        # DAM_indexes를 지정함
        for l in range(self.L):
            nonzero_DAM_l = np.nonzero(self.DAM[l])
            DAM_indexes_l = np.empty((1,len(nonzero_DAM_l[0])), dtype="int32")
            # l 레이어 차원의 수 만큼 각 차원에 대한 인덱스들을 DAM_indexes_l에 삽입
            for i in range(len(nonzero_DAM_l)):
                DAM_indexes_l = np.append(DAM_indexes_l, nonzero_DAM_l[i].reshape(1,-1), axis=0)
            # 처음 배열은 np.empty 메소드로 만들어진 쓰레기 값이라 버림
            # 가로 방향이라 세로 방향으로 길게 늘어지도록 바꿈
            DAM_indexes_l = list(DAM_indexes_l[1:].T)
            # DAM_indexes_l 각 원소가 리스트 형태인데 그것을 튜플로 바꿈
            # 튜플로 만드는 이유는 np.item() 메소드가 튜플을 인자로 받기 때문
            for i in range(len(DAM_indexes_l)):
                DAM_indexes_l[i] = tuple(DAM_indexes_l[i])

            self.DAM_indexes.append(DAM_indexes_l)
    
    # 아래의 방식을 쓴다면 거리 계산하기 전에 걸리는 시간도 길어지고 그다지 효율적이지도 않음.
    # 따라서 index_infos를 구하지 않음
    def set_index_infos(self):
        wtest = self.origin_names[2]

        for l in range(self.L):
            # mid_l, mean_l, std_l, min_l, max_l를 모두 -1로 초기화
            # 거리가 -1(음수)이 될 수 없으므로 -1인 부분은 초기화 되지 않은 인덱스 값임
            mid_l = np.zeros(self.shape[l]); mid_l = mid_l - 1
            mean_l = np.zeros(self.shape[l]); mean_l = mean_l - 1
            std_l = np.zeros(self.shape[l]); std_l = std_l - 1
            min_l = np.zeros(self.shape[l]); min_l = min_l - 1
            max_l = np.zeros(self.shape[l]); max_l = max_l - 1
            for DAM_index_l in self.DAM_indexes[l]:
                ED_array_l_DAM_index = np.empty((0), dtype="float")
                for k in range(self.origin_K[wtest]):
                    WFM_k_l = np.load(f"{self.wtest_dir}/{wtest}_{k}_{l}.npy")
                    # append를 많이 사용하면 속도가 줄어듦
                    ED_array_l_DAM_index = np.append(ED_array_l_DAM_index, abs(self.TFM_mean[l].item(DAM_index_l)
                                                                                 - WFM_k_l.item(DAM_index_l))) # TFM_mean = BFM

                mid_l.itemset(DAM_index_l, sorted(ED_array_l_DAM_index)[self.origin_K[wtest] // 2])
                mean_l.itemset(DAM_index_l, ED_array_l_DAM_index.mean())
                std_l.itemset(DAM_index_l, ED_array_l_DAM_index.std())
                min_l.itemset(DAM_index_l, ED_array_l_DAM_index.min())
                max_l.itemset(DAM_index_l, ED_array_l_DAM_index.max())

            self.mid.append(mid_l)
            self.mean.append(mean_l)
            self.std.append(std_l)
            self.min.append(min_l)
            self.max.append(max_l)
    # 사용하지 않음
    def md_lfmd(self, dir1, dir2, k, l):
        '''
        중앙값으로 나눠서 구한 레이어 거리(middle division layer feature map distance)

        dir1은 root 디렉토리 기준으로 깊이가 1인 것
        dir2은 root 디렉토리 기준으로 깊이가 2인 것
        k는 몇 번째 데이터인지 알려줌, l은 몇번째 레이어인지 알려줌
        예시: MDD("origin", "train", 1, 2)은
        root 디렉토리/origin/train/train_1_2에 대한 레이어 거리임.
        '''
        md_lfmd = 0
        dir2_FM_k_l = np.load(f'{self.root_dir}/{dir1}/{dir2}/{dir2}_{k}_{l}.npy')

        for index in self.DAM_indexes[l]:
            length_index = abs(self.TFM_mean[l].item(index) - dir2_FM_k_l.item(index))
            mid_index = self.mid[l].item(index)

            md_lfmd += (length_index / mid_index)**2
        
        return md_lfmd
    # 사용하지 않음
    def norm_lfmd(self, dir1, dir2, k, l):
        '''
        정규화를 시켜서 구한 레이어 거리(normal layer feature map distance)
        '''
        norm_lfmd = 0
        dir2_FM_k_l = np.load(f'{self.root_dir}/{dir1}/{dir2}/{dir2}_{k}_{l}.npy')
        
        for index in self.DAM_indexes[l]:
            length_index = abs(self.TFM_mean[l].item(index) - dir2_FM_k_l.item(index))
            mean_index = self.mean[l].item(index)
            std_index = self.std[l].item(index)

            norm_lfmd += math.exp((length_index - mean_index) / std_index)
        
        return norm_lfmd
    # 사용하지 않음
    def mms_lfmd(self, dir1, dir2, k, l):
        '''
        min-max으로 표준화한 후 shift하여 구한 레이어 거리(min max shift layer feature map distance)
        '''
        mms_lfmd = 0
        dir2_FM_k_l = np.load(f'{self.root_dir}/{dir1}/{dir2}/{dir2}_{k}_{l}.npy')
        
        for index in self.DAM_indexes[l]:
            length_index = abs(self.TFM_mean[l].item(index) - dir2_FM_k_l.item(index))
            mid_index = abs(self.TFM_mean[l].item(index) - self.mid[l].item(index))
            min_index = self.min[l].item(index)
            max_index = self.max[l].item(index)

            min_max_index = (length_index - min_index) / (max_index - min_index)
            min_max_mid_index = (mid_index - min_index) / (max_index - min_index)
            
            if length_index <= mid_index:
                mms_lfmd += (min_max_index + 0)**2
            else:
                mms_lfmd += (min_max_index + 1 - min_max_mid_index)**2
        
        return mms_lfmd
    
    def se_lfmd(self, FM_k_l, l, percent=50):
        '''
        se_lfmd: shift exponential layer feature map distance
        일단 디폴트로 length_min length_max의 50(정중앙 값)에 해당하는 부분을 origin(원점)으로 이동
        '''
        se_lfmd = 0
        
        # self.TFM_mean[l], FM_k_l를 norm_min, norm_max으로 정규화
        # norm_min과 norm_max을 1. 속성으로 두거나 2. 형식 매개변수로 둬도 되는데 3. 일단 지역 변수로 두자.
        # 이것도 [hyper parameter]가 될 수도 있겠다.
        norm_min = -1; norm_max = 1
        TFM_mean_l_norm = self.normalize_layer(self.TFM_mean[l], norm_min, norm_max)
        FM_k_l_norm = self.normalize_layer(FM_k_l, norm_min, norm_max)

        # lengths: 인덱스 마다 TFM_mean_norm과 FM_k_l_norm 사이의 거리(절대값)를 구함
        lengths = abs(TFM_mean_l_norm - FM_k_l_norm)
        
        # 'shift_value'을 구함
        # norm_min, norm_max가 같은 두 값의 길이의 min(length_min)은 0이고
        # length_max는 norm_max - norm_min임
        length_max = norm_max - norm_min; length_min = 0
        # 
        length_interval_max = length_max - length_min
        length_interval_percent = length_interval_max * (percent/100)
        # value_to_be_origin는 나중에 원점이 될 값임
        value_to_be_origin = length_min + length_interval_percent
        # shift_value는 value_to_be_origin을 0으로 옮기기 위한 이동 값임
        shift_value = -value_to_be_origin

        # 각 원소를 shift value 만큼 이동시키고 'exponential'을 취함
        for index in self.DAM_indexes[l]:
            exp_lengths_minus_shift_value = np.exp(lengths - shift_value)
        # se를 취한 값들을 모두 더한 것이 se_lfmd임
        se_lfmd = exp_lengths_minus_shift_value.sum()
        
        return se_lfmd
    
    def normalize_layer(self, layer, min, max):
        '''
        'layer의 min, max'으로 layer를 'min-max 정규화'를 한 후
        최소값이 min, 최대값이 max가 되도록 layer를 정규화 한다.
        layer(넘파이)에 스칼라 곱셈과 스칼라 덧셈을 적용하여 구현할 수 있다.
        '''
        # 'layer의 min, max'으로 layer를 'min-max 정규화'
        layer_min = layer.min(); layer_max = layer.max();
        # layer 값들이 하나라도 다르다면
        if layer_max - layer_min != 0:
            layer = (layer - layer_min) / (layer_max - layer_min)
        # layer 값들이 모두 같다면
        else:
            layer = 0

        scalar_multiplyer = max - min
        scalar_adder = min

        normalized_layer = scalar_multiplyer*layer + scalar_adder

        return normalized_layer
    
    def ab_lfmd(self, FM_k_l, l):
        '''
        ab_lfmd: absolute layer feature map distance
        레이어의 인덱스들 간의 절대값을 모두 더한다.
        '''
        ab_lfmd = 0
        
        # self.TFM_mean[l], FM_k_l를 norm_min, norm_max으로 정규화
        # norm_min과 norm_max을 1. 속성으로 두거나 2. 형식 매개변수로 둬도 되는데 3. 일단 지역 변수로 두자.
        # 이것도 [hyper parameter]가 될 수도 있겠다.
        norm_min = -1; norm_max = 1
        TFM_mean_l_norm = self.normalize_layer(self.TFM_mean[l], norm_min, norm_max)
        FM_k_l_norm = self.normalize_layer(FM_k_l, norm_min, norm_max)

        # lengths: 인덱스 마다 TFM_mean_norm과 FM_k_l_norm 사이의 거리(절대값)를 구함
        lengths = abs(TFM_mean_l_norm - FM_k_l_norm)
        
        # ed를 취한 값들을 모두 더한 것이 se_lfmd임
        ab_lfmd = lengths.sum()
        
        return ab_lfmd

    def fmd(self, FM_k):
        # 피처 맵 거리를 0으로 초기화
        fmd=0
        # lfmds: 레이어 피처 맵 거리를 담는 곳
        lfmds=[]
        # 각 레이어에 대한 레이어 피처 맵 거리 계산법으로 레이어 피처 맵 계산
        for l in range(self.L):
            FM_k_l = FM_k[l]
            # se_lfmd의 percent 인자의 디폴트 값이 50인데 이 다르게 하여 계산할 수도 있음
            lfmd_l = self.se_lfmd(FM_k_l, l)
            lfmds.append(lfmd_l)
        # 레이어 피처 맵마다 weight를 줌
        for l in range(self.L):
            fmd += self.W[l]*lfmds[l]

        return fmd
    
    def set_fmdc(self):
        # fmds는 오분류 테스트 데이터에 대한 fmd를 저장함
        fmds=[]

        # WFM을 부르기 위한 변수들을 선언함
        wtest=self.origin_names[2]; wtest_dir=self.wtest_dir; wtest_K=self.origin_K[wtest]
        WFMP_k=None; prev_WFMPI_k=None; cur_WFMPI_k=None;

        # k(=0)가 속한 WFMP을 가지고 옴
        k = 0
        prev_WFMPI_k = k // 1000
        with open(f'{wtest_dir}/{wtest}_{prev_WFMPI_k}.pickle', 'rb') as f:
            WFMP_k = pickle.load(f)

        # 오분류 데이터의 fmd의 최소값을 fmdc로 정함
        for k in range(wtest_K):
            # k번 째 데이터의 WFMPI, WFMPO 구함
            cur_WFMPI_k = k // 1000
            WFMPO_k = k % 1000

            # WFMP_k가 이미 있다면 가지고 오지 않고
            #             없다면 이전 WFMP를 지우고 현재 WFMP를 가지고 온다.

            # cur_WFMPI_k와 prev_WFMPI_k가 같다면
            if cur_WFMPI_k == prev_WFMPI_k:
                pass # 아무 작업 하지 않고

            # cur_WFMPI_k와 prev_WFMPI_k가 다를 경우
            else:
                # 이전 WFMP_k의 기억공간을 램에서 제거한 후
                del WFMP_k
                # cur_WFMPI_k를 현재 WFMP_k를 가지고 온다.
                with open(f'{wtest_dir}/{wtest}_{cur_WFMPI_k}.pickle', 'rb') as f:
                    WFMP_k = pickle.load(f)

            # prev_WFMPI_k를 cur_WFMPI_k로 초기화 
            prev_WFMPI_k = cur_WFMPI_k

            WFM_k = WFMP_k[WFMPO_k]

            fmds.append(self.fmd(WFM_k))
            
        fmds = np.array(fmds)
        
        self.fmdc = fmds.min()

    def eval(self):
        for eval_name in self.eval_names:
            # is_ts_fmds에 각 데이터마다 ts_fmd이라면 1, 아니라면 0이 저장됨
            is_ts_fmds = []
            fmds = []

            # EFM을 부르기 위한 변수들을 선언함
            eval_dir=f'{self.eval_dir}/{eval_name}'; eval_K=self.eval_K[eval_name]
            EFMP_k=None; prev_EFMPI_k=None; cur_EFMPI_k=None;

            # k(=0)가 속한 EFMP을 가지고 옴
            k = 0
            prev_EFMPI_k = k // 1000
            
            # {eval_name}_{prev_EFMPI_k}.pickle이 있는지 확인
            is_there_eval_name_prev_EFMPI_k_pickle = os.path.isfile(f'{eval_dir}/{eval_name}_{prev_EFMPI_k}.pickle')
            # {eval_name}_{prev_EFMPI_k}.pickle가 있다면 이 파일을 가지고 오고
            if is_there_eval_name_prev_EFMPI_k_pickle:
                with open(f'{eval_dir}/{eval_name}_{prev_EFMPI_k}.pickle', 'rb') as f:
                    EFMP_k = pickle.load(f)
            # {eval_name}_{prev_EFMPI_k}.pickle가 없다면
            # eval_{prev_EFMPI_k}.pickle 파일을 가지고 온다.
            else:
                with open(f'{eval_dir}/eval_{prev_EFMPI_k}.pickle', 'rb') as f:
                    EFMP_k = pickle.load(f)

            for k in range(1, eval_K):
                # k번 째 데이터의 EFMPI, EFMPO 구함
                cur_EFMPI_k = k // 1000
                EFMPO_k = k % 1000

                # EFMP_k가 이미 있다면 가지고 오지 않고
                #             없다면 이전 EFMP를 지우고 현재 EFMP를 가지고 온다.

                # cur_EFMPI_k와 prev_EFMPI_k가 같다면
                if cur_EFMPI_k == prev_EFMPI_k:
                    pass # 아무 작업 하지 않고

                # cur_EFMPI_k와 prev_EFMPI_k가 다를 경우
                else:
                    # 이전 EFMP_k의 기억공간을 램에서 제거한 후
                    del EFMP_k

                    # cur_EFMPI_k를 현재 EFMP_k를 가지고 온다.

                    # {eval_name}_{cur_EFMPI_k}.pickle이 있는지 확인
                    is_there_eval_name_cur_EFMPI_k_pickle = os.path.isfile(f'{eval_dir}/{eval_name}_{cur_EFMPI_k}.pickle')
                    # {eval_name}_{cur_EFMPI_k}.pickle가 있다면 이 파일을 가지고 오고
                    if is_there_eval_name_cur_EFMPI_k_pickle:
                        with open(f'{eval_dir}/{eval_name}_{cur_EFMPI_k}.pickle', 'rb') as f:
                            EFMP_k = pickle.load(f)
                    # {eval_name}_{cur_EFMPI_k}.pickle가 없다면
                    # eval_{cur_EFMPI_k}.pickle 파일을 가지고 온다.
                    else:
                        with open(f'{eval_dir}/eval_{cur_EFMPI_k}.pickle', 'rb') as f:
                            EFMP_k = pickle.load(f)

                # prev_EFMPI_k를 cur_EFMPI_k로 초기화 
                prev_EFMPI_k = cur_EFMPI_k

                EFM_k = EFMP_k[EFMPO_k]
                
                fmd = self.fmd(EFM_k)

                fmds.append(fmd)
                is_ts_fmds.append(fmd >= self.fmdc)
            
            fmds = np.array(fmds)
            is_ts_fmds = np.array(is_ts_fmds)

            # is_ts_fmds: 각 데이터가 ts_fmd인지 아닌지 알려주는 넘파이 배열
            np.save(f"{eval_dir}/{eval_name}_is_ts_fmds.npy",is_ts_fmds)
            # fmds: 각 데이터의 fmd를 저장하는 넘파이 배열
            np.save(f"{eval_dir}/{eval_name}_fmds.npy",fmds)
        
    def show_all(self):
        print('self.show_data_infos')
        self.show_data_infos()
        print('self.show_FM_means')
        self.show_FM_means()
        print('self.show_AMs_and_related')
        self.show_AMs_and_related()
        print('self.show_hyper_parameter')
        self.show_hyper_parameter()
        print('self.show_index_infos')
        '''
        사용하지 않음
        self.show_index_infos()
        print('self.show_layer_infos')
        '''
        self.show_layer_infos()
        print('self.show_fmdc')
        self.show_fmdc()
        print('self.show_dirs')
        self.show_dirs()
        print('self.show_eval_infos')
        self.show_eval_infos()
        
    def show_data_infos(self):
        print("self.origin_names"); print(self.origin_names)
        print("self.origin_K"); print(self.origin_K)
        print("self.eval_names"); print(self.eval_names)
        print("self.eval_K"); print(self.eval_K)
        print("self.L"); print(self.L)
        print("self.shape"); print(self.shape)

    def show_FM_means(self):
        print("self.TFM_mean"); self.show_square_FM(self.TFM_mean)
        print("self.RFM_mean"); self.show_square_FM(self.RFM_mean)
        print("self.WFM_mean"); self.show_square_FM(self.WFM_mean)

    def show_AMs_and_related(self):
        print("self.TAM"); self.show_square_FM(self.TAM)
        print("self.RAM"); self.show_square_FM(self.RAM)
        print("self.WAM"); self.show_square_FM(self.WAM)

        print("self.alpha_slice"); print(self.alpha_slice)
        print("self.alpha_min"); print(self.alpha_min)
        print("self.alpha"); print(self.alpha)
        print("self.alpha_max"); print(self.alpha_max)
        print("self.w_minus_r_max"); print(self.w_minus_r_max)

        print("self.DAM_select"); print(self.DAM_select)
        print("self.DAM"); self.show_square_FM(self.DAM)
        print("self.DAM_indexes"); print(self.DAM_indexes)

    def show_hyper_parameter(self):
        print("self.alpha_slice"); print(self.alpha_slice)
        print("self.DAM_select"); print(self.DAM_select)
        print("self.lfmd_select"); print(self.lfmd_select)
        print("self.W"); print(self.W)

    def show_index_infos(self):
        print("self.mid"); self.show_square_FM(self.mid)
        print("self.mean"); self.show_square_FM(self.mean)
        print("self.std"); self.show_square_FM(self.std)
        print("self.min"); self.show_square_FM(self.min)
        print("self.max"); self.show_square_FM(self.max)

    def show_layer_infos(self):
        print("self.lfmd_select"); print(self.lfmd_select)
        print("self.W"); print(self.W)

    def show_fmdc(self):
        print("self.fmdc"); print(self.fmdc)

    def show_dirs(self):
        print("self.root_dir"); print(self.root_dir)
        
        print("self.origin_dir"); print(self.origin_dir)
        print("self.train_dir"); print(self.train_dir)
        print("self.rtest_dir"); print(self.rtest_dir)
        print("self.wtest_dir"); print(self.wtest_dir)

        print("self.eval_dir"); print(self.eval_dir)

    def show_eval_infos(self):
        for eval_name in self.eval_names:
            print(f'In eval/{eval_name}')
            
            result_npy = np.load(f'{self.eval_dir}/{eval_name}/{eval_name}_result.npy')
            print(f'{eval_name}_result.npy'); print(result_npy)
            
            is_ts_fmds_npy = np.load(f'{self.eval_dir}/{eval_name}/{eval_name}_is_ts_fmds.npy')
            print(f'{eval_name}_is_ts_fmds.npy'); print(is_ts_fmds_npy)
            
            fmds_npy = np.load(f'{self.eval_dir}/{eval_name}/{eval_name}_fmds.npy')
            print(f'{eval_name}_fmds.npy'); print(fmds_npy)
    
    def show_square_FM(self, np_arr):
        # np_arr에 np_arr의 최소의 값부터 최대 값까지 그리는 넘파이 추가
        # 이 넘파이 배열을 그리는 이유는 최소 최대에 대한 시각적인 표현을 할 수 있을 뿐만 아니라
        # color bar로 인해 좁게 그려지는 넘파이 배열을 없앨 수 있다.
        # np_arr_min 찾기
        np_arr_min = np_arr[0].min()
        for i in range(1, len(np_arr)):
            if np_arr_min > np_arr[i].min():
                np_arr_min = np_arr[i].min()
        # np_arr_max 찾기
        np_arr_max = np_arr[0].max()
        for i in range(1, len(np_arr)):
            if np_arr_max > np_arr[i].max():
                np_arr_max = np_arr[i].max()
        # bool 타입이라면 마지막에 넘파이에 True, False만 넣고 즉, (T,T), (T,F), (F,T), (F,F)만 넣고
        if str(np_arr[0].dtype) == 'bool':
            np_arr.append(np.array([np_arr_min, np_arr_max]))
        # 그게 아니라면 마지막에 넘파이에 np_arr_slice로 잘린 연속적인 값들을 넣음.
        else:
            # np_arr_slice: np_arr_min 부터 np_arr_max 까지 그리는데 몇 번에 걸쳐서 그릴지 정하기
            np_arr_slice = 1000
            np_arr_interval = (np_arr_max - np_arr_min) / np_arr_slice
            # np_arr에 np_arr의 최소의 값부터 최대 값까지 그리는 넘파이 생성
            np_min_to_max = np.array([np_arr_min + i*np_arr_interval for i in range(np_arr_slice+1)])
            np_arr.append(np_min_to_max)

        # 레이어 개수 만큼 레이어 원소 개수 계산
        element_count=[]
        for ith in range(len(np_arr)):
            ith_element_count = 1
            for ith_shape_ele in np_arr[ith].shape:
                ith_element_count *= ith_shape_ele
            
            element_count.append(ith_element_count)
        # 레이어 원소 개수로 정방 이차 배열의 한 변의 길이 구함
        square_FM_side=[]
        for ith in range(len(np_arr)):
            square_FM_side.append(math.ceil(math.sqrt((element_count[ith]))))
        # square_FM_side_min으로 그래프 크기를 정함
        square_FM_side_min = np.array(square_FM_side).min()
        square_FM_side_max = np.array(square_FM_side).max()

        # 레이어 피처 맵을 평탄화함.
        np_arr_flatten=[]
        for ith in range(len(np_arr)):
            np_arr_flatten.append(np_arr[ith].flatten())

        # x,y로 np_arr를 그리기 위한 이차 정방 배열 좌표를 만듦
        x=[];y=[]
        for ith in range(len(np_arr)):
            x_ith=[]; y_ith=[]
            for y_ in range(square_FM_side[ith]):
                for x_ in range(square_FM_side[ith]):
                    x_ith.append(x_)
                    y_ith.append(y_)
            # x_ith, y_ith를 np_arr[ith] 개수 만큼 자름
            x_ith = x_ith[:element_count[ith]]; y_ith = y_ith[:element_count[ith]]
            # x_ith을 x에 넣고 y_ith을 y에 넣음
            x.append(x_ith); y.append(y_ith)

        # plt의 subplot의 크기 지정
        plt_column = 15
        plt_row = (len(np_arr)-1 // plt_column) + 1

        # 넘파이 배열을 5줄 씩 그리기
        plt.figure(figsize=(3*20, 8*20))
        for i in range(len(np_arr)):
            plt.subplot(plt_row,plt_column,i+1)
            plt.scatter(x=x[i], y=y[i], c=np_arr_flatten[i], cmap='jet')
        plt.colorbar()

    def save(self, model_name):
        # 인자로 지정된 경로와 이름으로 파일 저장
        with open(f"./instances/{model_name}.pickle", "wb") as f:
            pickle.dump(self, f, pickle.HIGHEST_PROTOCOL)

    def load(self, model_name):
        # 인자로 지정된 경로와 이름으로 파일 불러오기
        with open(f"./instances/{model_name}.pickle", "rb" ) as f:
            return copy.deepcopy(pickle.load(f))

In [ ]:
fmd1 = FMD("/Volumes/My Passport_ssd_sg3/data_sets/fmnist/shirts")
# fmd1.create_practice()
fmd1.fit()

In [ ]:
fmd1.show_fmdc()

self.fmdc
4199.252005996141


In [ ]:
fmd1.eval()

In [ ]:
fmd1.show_eval_infos()

In eval/rotation_90
rotation_90_result.npy
[0. 0. 0. ... 0. 0. 1.]
rotation_90_is_ts_fmds.npy
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True

brightness_10_result.npy
[0. 0. 0. ... 0. 0. 1.]
brightness_10_is_ts_fmds.npy
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  Tr

whiteness_10_result.npy
[0. 0. 0. ... 0. 0. 1.]
whiteness_10_is_ts_fmds.npy
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
 False  True  True  True  True  True False  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True

In [ ]:
a = np.load('/Volumes/My Passport_ssd_sg3/data_sets/fmnist/shirts/eval/rotation_90/rotation_90_result.npy')

In [ ]:
print(len(a))
print(len(np.nonzero(a)[0]))
print(len(a)-len(np.nonzero(a)[0]))

47040001
23423503
23616498


In [ ]:
start = time.process_time()

with open(f'/Volumes/My Passport_ssd_sg3/data_sets/fmnist/shirts/eval/brightness_10/brightness_10_0.pickle', 'rb') as f:
    EFMP_k = pickle.load(f)

end = time.process_time()
print("Time elapsed: ", timedelta(seconds=end-start))
    
start = time.process_time()

a = EFMP_k[0]

end = time.process_time()
print("Time elapsed: ", timedelta(seconds=end-start))

Time elapsed:  0:00:03.245296
Time elapsed:  0:00:00.063163
